In [ ]:
import os
from dotenv import load_dotenv
from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockTradesRequest
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
load_dotenv()
api_key = os.getenv("ALPACA_API_KEY")
secret_key = os.getenv("ALPACA_SECRET_KEY")


client = StockHistoricalDataClient(api_key, secret_key)
request = StockTradesRequest(symbol_or_symbols="ANET", start=datetime(2026, 1, 15), end=datetime(2026, 1, 16))
trades = client.get_stock_trades(request)

In [ ]:
df = trades.df
print(f"Downloaded {len(df)} trades")
print(df.head(10))
print(df.info())

In [ ]:
df.to_csv("anet_exec_data.csv")

In [ ]:
df_reset = df.reset_index(level='symbol', drop=True)

df_5min = df_reset.resample('5min').agg({
    'price': ['first', 'max', 'min', 'last'],
    'size': 'sum'
})

df_5min.columns = ['open', 'high', 'low', 'close', 'volume']

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

ax1.plot(df_5min.index, df_5min['close'], label='Close', linewidth=1.5, color='blue')
ax1.fill_between(df_5min.index, df_5min['low'], df_5min['high'], alpha=0.3, color='lightblue')
ax1.set_ylabel('Price ($)')
ax1.set_title('ANET 5-Minute Price Action - Jan 15, 2026')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.bar(df_5min.index, df_5min['volume'], width=0.003, alpha=0.7, color='steelblue')
ax2.set_xlabel('Time')
ax2.set_ylabel('Volume (shares)')
ax2.set_title('5-Minute Trading Volume')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
df = pd.read_csv('anet_exec_data.csv', parse_dates=['timestamp'], index_col=['symbol', 'timestamp'])
df_reset = df.reset_index(level='symbol', drop=True)

exchange_volume = df_reset.groupby('exchange')['size'].sum().sort_values(ascending=False)

plt.figure(figsize=(12, 6))
plt.bar(exchange_volume.index, exchange_volume.values, color='steelblue', edgecolor='black')
plt.xlabel('Exchange')
plt.ylabel('Total Volume (shares)')
plt.title('Trading Volume by Exchange - ANET, Jan 15, 2026')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print(f"Total volume: {exchange_volume.sum():,.0f} shares")
print(f"\nTop 3 exchanges:") # N-NYSE, D-FINRA ADF, T-Nasdaq TRF, V-IEX
for exchange, volume in exchange_volume.head(3).items():
    pct = (volume / exchange_volume.sum() * 100)
    print(f"{exchange}: {volume:,.0f} shares ({pct:.1f}%)")

In [ ]:
import numpy as np

from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockTradesRequest

daily_revenue = btc_mined * btc_price

window = 365
weights = np.ones(window) / window
ma_365 = np.convolve(daily_revenue, weights, mode="full")[: len(daily_revenue)]
ma_365[: window-1] = np.nan

puell_multiple = daily_revenue / ma_365

In [1]:
import os
from dotenv import load_dotenv
from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockTradesRequest
from datetime import datetime
import pandas as pd

load_dotenv()
api_key = os.getenv("ALPACA_API_KEY")
secret_key = os.getenv("ALPACA_SECRET_KEY")

client = StockHistoricalDataClient(api_key, secret_key)

nasdaq_top_50 = [
    "NVDA", "MU", "GOOGL","SOFI","INTC","TSLA","AAPL","PLTR","META","AMD"
]

# nasdaq_top_50 = [
#     "NVDA", "MU", "GOOGL", "INTC", "TSLA", "AAPL", "PLTR", "AMD", "META", "MSFT",
#     "AMZN", "GOOG", "AVGO", "NFLX", "QCOM", "SOFI", "HOOD", "GME", "MARA", "MSTR",
#     "COIN", "AMAT", "LRCX", "KLAC", "MRVL", "SNPS", "CDNS", "ASML", "CSCO", "ADBE",
#     "CRM", "ORCL", "RIVN", "LCID", "NIO", "MRNA", "BNTX", "NVAX", "BABA", "PDD",
#     "JD", "BIDU", "CMCSA", "PYPL", "SBUX", "COST", "ABNB", "ON", "MPWR", "SMCI"
# ]

request = StockTradesRequest(symbol_or_symbols=nasdaq_top_50,
    start=datetime(2026, 2, 4),
    end=datetime(2026, 2, 5)
)

trades = client.get_stock_trades(request)
df = trades.df.reset_index(level='symbol', drop=False)

df.to_csv('nasdaq_top50_trades.csv')
print(f"Downloaded {len(df):,} trades for {len(nasdaq_top_50)} stocks")
print("Trades per symbol:")
print(df.groupby('symbol').size().sort_values(ascending=False))

/Users/tobiasmanwaring/Execution-CNN/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Downloaded 13,965,365 trades for 10 stocks
Trades per symbol:
symbol
NVDA     3258064
PLTR     1807056
TSLA     1794918
AMD      1672254
GOOGL    1534127
AAPL     1099187
MU       1082065
INTC      777104
META      489219
SOFI      451371
dtype: int64
